# Transfer Learning Example

## Transfer learning is a basic appoach of model reuse and retraining
### A model trained on one dataset for a different domain is refined by modifying some of the last layers and trained with new dataset
    * This saves a lot of training time as we only need to modify some of the layers and retrain only those layers
    * Also sometimes we don't have a very big dataset which we can use for training a model so we take pretrained model and retrain it by making only some of the layers trainable
    * This is one of the basic techniques for domain adaptation

### This is a basic example from keras examples directory
(Available @ https://github.com/keras-team/keras/blob/master/examples/mnist_transfer_cnn.py)

    * - Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
    * - Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].
   

Get to 99.8% test accuracy after 5 epochs
for the first five digits classifier
and 99.2% for the last five digits after transfer + fine-tuning.

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [2]:
now = datetime.datetime.now

batch_size = 128  # no.of elements to be used for one iteration
num_classes = 5   # no. of classes for training
epochs = 5        # how many times the whole dataset should be iterated

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3  # here kernel_size means a 3x3 filter

if K.image_data_format() == 'channels_first':  # channels mean no. of color channels of the image
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)    # tensorflow uses channels_last config by default

### Define the funtion which will run the training with input model and training data
    This function basically does some preprocessing on training data and then runs compile and fit functions of keras.models.Sequential

In [3]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    # compile the model
    # you can chnage the parameters in this compile function
    # custom funtions for loss and opitizer can be used: ref to keras documentation for more
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()

    # Train the model
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [4]:
# Get the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:

# create two datasets
# one with digits below 5
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]
print(x_train.shape)
print(x_train[y_train<5].shape)


(60000, 28, 28)
(30596, 28, 28)


In [6]:
# one with digits below 5 one with 5 and above

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [7]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# create complete model
model = Sequential(feature_layers + classification_layers)


In [9]:

# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 53s 213ms/step - accuracy: 0.2377 - loss: 1.5978 - val_accuracy: 0.3514 - val_loss: 1.5739
Epoch 2/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 80s 206ms/step - accuracy: 0.3188 - loss: 1.5737 - val_accuracy: 0.4472 - val_loss: 1.5454
Epoch 3/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 84s 214ms/step - accuracy: 0.4015 - loss: 1.5468 - val_accuracy: 0.5221 - val_loss: 1.5127
Epoch 4/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 80s 207ms/step - accuracy: 0.4554 - loss: 1.5154 - val_accuracy: 0.5877 - val_loss: 1.4741
Epoch 5/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 84s 216ms/step - accuracy: 0.5086 - loss: 1.4815 - val_accuracy: 0.6737 - val_loss: 1.4289
Training time: 0:06:21.425220
Test score: 1.4288829565048218
Test accuracy: 0.6736719012260437


### Model trained in the above block can be used for classifying digits 5 to 9 by fine tuning it
    For fine tuning we will freeze all the convolutional and maxpooling layers (feature layers)
    This can be done by making those layers non-trainable
    only the top(last) two layers (dense layers) are left trainable

In [10]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False


In [11]:

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 21s 82ms/step - accuracy: 0.2359 - loss: 1.6055 - val_accuracy: 0.3304 - val_loss: 1.5830
Epoch 2/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 18s 78ms/step - accuracy: 0.2855 - loss: 1.5833 - val_accuracy: 0.4454 - val_loss: 1.5601
Epoch 3/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 19s 72ms/step - accuracy: 0.3348 - loss: 1.5623 - val_accuracy: 0.5521 - val_loss: 1.5377
Epoch 4/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 22s 78ms/step - accuracy: 0.3830 - loss: 1.5415 - val_accuracy: 0.6367 - val_loss: 1.5155
Epoch 5/5
230/230 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.4411 - loss: 1.5214 - val_accuracy: 0.7015 - val_loss: 1.4934
Training time: 0:01:40.156957
Test score: 1.4933886528015137
Test accuracy: 0.701501727104187
